In [67]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/stocks/gr500182.csv
/kaggle/input/stocks/gr500209.csv
/kaggle/input/stocks/gr500680.csv
/kaggle/input/stocks/gr530965.csv
/kaggle/input/stocks/gr532174.csv
/kaggle/input/stocks/gr532210.csv
/kaggle/input/stocks/gr532540.csv
/kaggle/input/stocks/gr500325.csv
/kaggle/input/stocks/gr500112.csv
/kaggle/input/stocks/gr500180.csv
/kaggle/input/stocks/gr507685.csv


In [68]:
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from math import sqrt
from sklearn.neighbors import DistanceMetric
from sklearn.model_selection import GridSearchCV
import prettytable


In [69]:
import warnings; warnings.simplefilter('ignore')

In [70]:
def pre_process_data(data,null_threshold):
    """
    Drops Date and Unix Date columns from the data.
    Drops the columns which has null values more than specified null_threshold.
    Replaces infinite values with NAN.
    Drops the rows which has null values.

    Parameters
    ----------
    data : dataframe

    null_threshold : numeric
        numeric value describing the amount of null values that can be present.

    Returns
    -------
    data : dataframe
        an updated dataframe after performing all the opertaions.
    """
    
    data.drop(columns=['Unix Date','Date'],axis=1,inplace=True)
    total = data.shape[0]
    for col in data.columns:
        if null_threshold * total / 100 < data[col].isnull().sum():
            data.drop(columns=[col],axis=1,inplace=True)
    data.replace([np.inf, -np.inf], np.nan, inplace=True)
    data = data.apply(pd.to_numeric,errors='coerce')
    data.dropna(axis=0,inplace=True)
    return data

In [71]:
def dependent_column(data,column):
    """
    Removes all the Next Day columns.
    Removes all the non Growth Rate Columns (GR)
    add the predictor column to list of columns.

    Parameters
    ----------
    data : dataframe

    column : string
        name of the predictor column 

    Returns
    -------
    data : dataframe
        an updated dataframe after performing all the opertaions.
    column : string
        name of the predictor column
    """
    cols = [col for col in data.columns if "next" not in col.lower() and col.lower().endswith("gr")]
    cols.append(column)
    data = data[cols]
    return (data,column)

In [72]:
def best_parameters(X,Y):
    params = {'n_neighbors':np.arange(1,105,5), 'weights':['uniform', 'distance'], 'metric':['euclidean', 'manhattan']}
    knn = KNeighborsClassifier()    
    model = GridSearchCV(knn, params)
    model.fit(X,Y)
    k = model.best_params_['n_neighbors']
    params = {'n_neighbors':np.arange(k-5, k+5), 'weights':['uniform', 'distance'], 'metric':['euclidean', 'manhattan']}
    knn = KNeighborsClassifier()
    model = GridSearchCV(knn, params)
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=0)
    model.fit(X_train,y_train)
    return model.best_params_

In [73]:
def create_confusion_matrix(y_pred,y_true):
    
    cm = confusion_matrix(y_true,y_pred)
    accuracy = metrics.accuracy_score(y_true,y_pred)
    precision = metrics.precision_score(y_true,y_pred)
    recall = metrics.recall_score(y_true,y_pred)
    f1_score = metrics.f1_score(y_true,y_pred)
    return {"accuracy":accuracy,"precision":precision,"recall":recall,"f1_score":f1_score,"confusion matrix":cm}

In [74]:
def k_nearest_neighbours_classification(df,column="Next Day Close Price GR",threshold = 0.01):
    df["Target"] = df[column].apply(lambda x : 1 if x >= threshold else 0)
    X = df.drop(columns=["Target",column])
    Y = df["Target"]
    
    params = best_parameters(X,Y)
    
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=0)
    knn = KNeighborsClassifier(**params)
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)
    
    myres = create_confusion_matrix(y_pred,y_test)
    
    rmse = sqrt(metrics.mean_squared_error(y_test, y_pred))
    mae = metrics.mean_absolute_error(y_test, y_pred)
    mse = metrics.mean_squared_error(y_test, y_pred)
    r2 = metrics.r2_score(y_test, y_pred)
    
    myres.update({"root_mean_squared_error":rmse,"mean_absolute_error":mae,"mean_squared_error":mse,"rsquared_adj":r2})
    myres.update(params)
    myres.update({'threshold':threshold})
    return myres

In [75]:
%%time
mydf = pd.DataFrame()
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        filepath = os.path.join(dirname, filename)
        df = pd.read_csv(filepath)
        df = pre_process_data(df,60)
        column = "Next Day Close Price GR"
        (df,column) = dependent_column(df,column)
        for i in [0.005,0.01,0.02,0.03,0.04,0.05]:
            result = k_nearest_neighbours_classification(df,threshold = i)
            result.update({"security id":filename[2:8]})
            mydf = mydf.append(result,ignore_index=True)
mydf.to_csv(os.path.join(os.getcwd(),"knn_class"+".csv"),index=None)

CPU times: user 33min 44s, sys: 1.2 s, total: 33min 45s
Wall time: 33min 49s


In [79]:
columns = ['security id','threshold','metric','n_neighbors','weights','precision','recall','f1_score','accuracy']
mydf = mydf[columns]
table = prettytable.PrettyTable()
table.title = "knn class"
table.field_names = columns
for _,row in mydf.iterrows():
    row = [round(r,6) if isinstance(r,(float,int)) else r for r in row]
    table.add_row(row)
print(table)